# Lab 2
### Library fft() example

In [1]:
%use kmath
import kotlinx.coroutines.Dispatchers
import kotlinx.coroutines.flow.toList
import kotlinx.coroutines.runBlocking
import org.apache.commons.math3.transform.TransformType
import space.kscience.kmath.streaming.asFlow
import space.kscience.kmath.commons.transform.fft

val frequency = 50.0
val w = 2 * PI * frequency
fun signal(x: Double) = cos(w * x)

val partsCount = 2048
val range = 0.0 to 10.0
val step = (range.second - range.first) / partsCount
val signalValues = (0..<partsCount).map { range.first + step * it }

val signalFlow = ListBuffer(signalValues).asFlow()
val complexAmplitudes = runBlocking(context = Dispatchers.Default) { 
    signalFlow.fft(bufferSize = partsCount, direction = TransformType.FORWARD).toList()
}

complexAmplitudes

[(10235.0 + i * 0.0), (-5.000000000000005 + i * 3259.4906778869695), (-5.000000000000027 + i * 1629.741503988507), (-5.000000000000086 + i * 1086.490074925904), (-4.999999999999859 + i * 814.8630820662498), (-4.999999999999946 + i * 651.8858636810376), (-4.999999999999868 + i * 543.233532525826), (-4.9999999999999964 + i * 465.6239940659092), (-4.999999999999981 + i * 407.41620103273095), (-4.999999999999931 + i * 362.1429049426891), (-5.000000000000088 + i * 325.9237567046607), (-5.000000000000088 + i * 296.28944325978273), (-5.000000000000083 + i * 271.59375590134374), (-5.000000000000095 + i * 250.69701167780354), (-5.0000000000002665 + i * 232.78515133737358), (-5.000000000000549 + i * 217.261198094767), (-4.999999999999962 + i * 203.67741936041656), (-4.99999999999991 + i * 191.6914313887926), (-4.999999999999895 + i * 181.03693571061447), (-4.999999999999877 + i * 171.5036966917326), (-4.999999999999881 + i * 162.92352582434526), (-4.999999999999874 + i * 155.1602705064401), (-4.

### dftSlow() example

In [2]:
import fourier.dftSlow
val complexAmplitudesDftSlow = dftSlow(signalValues)

kotlin.NotImplementedError: An operation is not implemented: Not yet implemented